In [ ]:
# Imports and file paths
import sys
import os
print('Python version:', sys.version)

# Standard data science libraries
import numpy as np
import pandas as pd
from pathlib import Path

# sklearn & joblib will be used
import sklearn
import joblib
print('pandas', pd.__version__, 'sklearn', sklearn.__version__)

# Paths (these files were uploaded in this session)
TRAIN_CSV = "/content/SENG_4610_Training_Data.csv"
FEATURES_CSV = "/content/Feature_Description.csv"

# Check files
print('Training CSV exists:', Path(TRAIN_CSV).exists())
print('Feature description exists:', Path(FEATURES_CSV).exists())

Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
pandas 2.2.2 sklearn 1.6.1
Training CSV exists: True
Feature description exists: True


In [ ]:
# Load CSVs
df = pd.read_csv(TRAIN_CSV, encoding = 'latin-1')
feat = pd.read_csv(FEATURES_CSV,  encoding = 'latin-1')

print('Training data shape:', df.shape)
display(df.head(10))
print('\n---\nFeature description preview:')
display(feat.head(20))




Training data shape: (175341, 45)


,ï»¿id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,...,1,1,0,0,0,1,1,0,Normal,0
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,...,1,2,0,0,0,1,6,0,Normal,0
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,...,1,3,0,0,0,2,6,0,Normal,0
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,1,3,1,1,0,2,1,0,Normal,0
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,...,1,40,0,0,0,2,39,0,Normal,0
5,6,0.380537,tcp,-,FIN,10,6,534,268,39.417980,...,1,40,0,0,0,2,39,0,Normal,0
6,7,0.637109,tcp,-,FIN,10,8,534,354,26.683033,...,1,40,0,0,0,1,39,0,Normal,0
7,8,0.521584,tcp,-,FIN,10,8,534,354,32.593026,...,1,40,0,0,0,3,39,0,Normal,0
8,9,0.542905,tcp,-,FIN,10,8,534,354,31.313031,...,1,40,0,0,0,3,39,0,Normal,0
9,10,0.258687,tcp,-,FIN,10,6,534,268,57.985135,...,1,40,0,0,0,3,39,0,Normal,0



---
Feature description preview:


,No.,Name,Type,Description
0,1,srcip,nominal,Source IP address
1,2,sport,integer,Source port number
2,3,dstip,nominal,Destination IP address
3,4,dsport,integer,Destination port number
4,5,proto,nominal,Transaction protocol
5,6,state,nominal,Indicates to the state and its dependent proto...
6,7,dur,Float,Record total duration
7,8,sbytes,Integer,Source to destination transaction bytes
8,9,dbytes,Integer,Destination to source transaction bytes
9,10,sttl,Integer,Source to destination time to live value


In [ ]:
# Quick EDA
print('Columns:', df.columns.tolist())
print('\nDtypes:\n', df.dtypes.value_counts())

# Missing values summary
missing = df.isna().sum().sort_values(ascending=False)
display(missing.head(30))

Columns: ['ï»¿id', 'dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth', 'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm', 'ct_srv_dst', 'is_sm_ips_ports', 'attack_cat', 'label']

Dtypes:
 int64      30
float64    11
object      4
Name: count, dtype: int64


,0
ï»¿id,0
dur,0
proto,0
service,0
state,0
spkts,0
dpkts,0
sbytes,0
dbytes,0
rate,0


In [ ]:
# Confirm target column names (adjust if different)
possible_label_cols = ['label', 'Label', 'attack', 'is_attack']
possible_attack_cat = ['attack_cat', 'attack_catagory', 'attack_category', 'attack_type']

print('Columns present that match common names:')
print([c for c in df.columns if c in possible_label_cols])
print([c for c in df.columns if c in possible_attack_cat])

# If your dataset uses different names, update these variables:
BINARY_TARGET = 'label'           # 0 normal, 1 attack
MULTI_TARGET = 'attack_cat'       # multiclass attack category
ID_COL = 'id'

# Drop ID if present
if ID_COL in df.columns:
    df = df.drop(columns=[ID_COL])
    print('Dropped id column')

Columns present that match common names:
['label']
['attack_cat']


In [ ]:
# Clean service & other textual columns
for col in ['service', 'proto', 'state']:
    if col in df.columns:
        df[col] = df[col].astype(str)  # convert to string first to avoid unexpected dtypes
        df[col] = df[col].replace(['-', '?', 'None', 'nan', ''], 'unknown')
        df[col] = df[col].fillna('unknown')

# Show unique counts
for col in ['service', 'proto', 'state']:
    if col in df.columns:
        print(f"-> {col}: unique count =", df[col].nunique(), 'example values =', df[col].unique()[:10])


-> service: unique count = 13 example values = ['unknown' 'ftp' 'smtp' 'snmp' 'http' 'ftp-data' 'dns' 'ssh' 'radius'
 'pop3']
-> proto: unique count = 133 example values = ['tcp' 'udp' 'arp' 'ospf' 'icmp' 'igmp' 'rtp' 'ddp' 'ipv6-frag' 'cftp']
-> state: unique count = 9 example values = ['FIN' 'INT' 'CON' 'ECO' 'REQ' 'RST' 'PAR' 'URN' 'no']


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Ensure binary target is numeric 0/1
if df[BINARY_TARGET].dtype == object:
    # if the label is text like 'Normal'/'Attack' or 'normal'/'anomaly'
    df['is_attack'] = df[BINARY_TARGET].str.lower().isin(['1','attack','anomaly','attack','attack!','attack?']).astype(int)
else:
    df['is_attack'] = df[BINARY_TARGET].astype(int)

print('is_attack distribution:')
print(df['is_attack'].value_counts())

# Encode multiclass only when present
if MULTI_TARGET in df.columns:
    le_attack = LabelEncoder()
    df['attack_cat_encoded'] = df[MULTI_TARGET].fillna('unknown').astype(str)
    df['attack_cat_encoded'] = le_attack.fit_transform(df['attack_cat_encoded'])
    print('\nMulticlass mapping (first 20):')
    mapping = dict(zip(le_attack.classes_, le_attack.transform(le_attack.classes_)))
    print(mapping)
else:
    print('No multiclass attack column found; will skip softmax training unless you update MULTI_TARGET.')

print(df['attack_cat'].value_counts())

is_attack distribution:
is_attack
1    119341
0     56000
Name: count, dtype: int64

Multiclass mapping (first 20):
{'Analysis': np.int64(0), 'Backdoor': np.int64(1), 'DoS': np.int64(2), 'Exploits': np.int64(3), 'Fuzzers': np.int64(4), 'Generic': np.int64(5), 'Normal': np.int64(6), 'Reconnaissance': np.int64(7), 'Shellcode': np.int64(8), 'Worms': np.int64(9)}
attack_cat
Normal            56000
Generic           40000
Exploits          33393
Fuzzers           18184
DoS               12264
Reconnaissance    10491
Analysis           2000
Backdoor           1746
Shellcode          1133
Worms               130
Name: count, dtype: int64


In [ ]:
# By default drop IPs and timestamp-like fields to avoid leakage and high-cardinality
drop_cols = []
for cand in ['srcip', 'dstip', 'Stime', 'Ltime', 'stime', 'ltime', 'stime', 'stime_ms']:
    if cand in df.columns:
        drop_cols.append(cand)

print('Initial drop_cols:', drop_cols)

# Numeric columns detection
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
# exclude target columns
exclude = ['is_attack', 'attack_cat_encoded', BINARY_TARGET]
numeric_cols = [c for c in numeric_cols if c not in exclude]
print('\nNumeric candidate columns (sample):', numeric_cols[:20])

# Categorical columns to encode
cat_cols = [c for c in ['service', 'proto', 'state'] if c in df.columns]
print('\nCategorical columns to encode:', cat_cols)

# Final features list — for pipeline we'll handle encoding/scaling
# Keep numeric_cols and cat_cols; remove drop_cols if present
features = [c for c in numeric_cols + cat_cols if c not in drop_cols]
print('\nFinal features to be used (sample):', features[:40])


Initial drop_cols: []

Numeric candidate columns (sample): ['ï»¿id', 'dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb']

Categorical columns to encode: ['service', 'proto', 'state']

Final features to be used (sample): ['ï»¿id', 'dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth', 'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm', 'ct_srv_dst', 'is_sm_ips_ports']


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support

# Define preprocessing steps
num_features = [c for c in features if c in df.columns and np.issubdtype(df[c].dtype, np.number)]
cat_features = [c for c in features if c in df.columns and c not in num_features]

num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, num_features),
    ('cat', cat_transformer, cat_features)
], remainder='drop')

print('num_features count:', len(num_features))
print('cat_features count:', len(cat_features))

# Prepare data for binary classification
X = df[features]
y = df['is_attack']

# Stratified split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)


num_features count: 40
cat_features count: 3


In [ ]:
from sklearn.neural_network import MLPClassifier

# Neural network pipeline (same preprocessing)
mlp_model = Pipeline(steps=[
('preproc', preprocessor),
('clf', MLPClassifier(
hidden_layer_sizes=(64, 32), # example: 2 layers, 64 and 32 neurons
activation='relu', # common choice
solver='adam', # adaptive optimizer
max_iter=200, # can increase if not converging
random_state=42,
early_stopping=True, # stop if validation score doesn't improve
verbose=True # shows progress
))
])
print('Training MLPClassifier...')
mlp_model.fit(X_train, y_train)
print('Done.')


Training MLPClassifier...
Iteration 1, loss = 0.14329366
Validation score: 0.948318
Iteration 2, loss = 0.10062198
Validation score: 0.955660
Iteration 3, loss = 0.09325353
Validation score: 0.956658
Iteration 4, loss = 0.08567800
Validation score: 0.964357
Iteration 5, loss = 0.07821420
Validation score: 0.967707
Iteration 6, loss = 0.07155414
Validation score: 0.967137
Iteration 7, loss = 0.06597820
Validation score: 0.969276
Iteration 8, loss = 0.06168312
Validation score: 0.972840
Iteration 9, loss = 0.05893828
Validation score: 0.972555
Iteration 10, loss = 0.05705680
Validation score: 0.972840
Iteration 11, loss = 0.05520030
Validation score: 0.972198
Iteration 12, loss = 0.05371209
Validation score: 0.976618
Iteration 13, loss = 0.05292465
Validation score: 0.973339
Iteration 14, loss = 0.05143733
Validation score: 0.972626
Iteration 15, loss = 0.05097852
Validation score: 0.975335
Iteration 16, loss = 0.04912985
Validation score: 0.974907
Iteration 17, loss = 0.04894657
Validat

In [ ]:
y_pred_nn = mlp_model.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix
print('\nClassification report (MLP binary):')
print(classification_report(y_test, y_pred_nn, digits=4))
print('\nConfusion matrix:')
print(confusion_matrix(y_test, y_pred_nn))


# hyperparameter to note of
# hidden_layer_sizes: start small (64,32) or (100,) to avoid overfitting.
# activation: 'relu' works well; 'tanh' or 'logistic' are alternatives.
# solver: 'adam' is usually robust; 'sgd' can be slower.
# early_stopping=True avoids wasting time on overfitting.
# max_iter: 200–500 is typical for small to medium datasets.


# Saving the model
import joblib
joblib.dump(mlp_model, 'mlp_attack_model.joblib')
# Later: loaded_model = joblib.load('mlp_attack_model.joblib')

# took less than a second to predict => time wise and coputation intensity (Good for hand held devices)


Classification report (MLP binary):
              precision    recall  f1-score   support

           0     0.9785    0.9685    0.9735     11200
           1     0.9853    0.9900    0.9876     23869

    accuracy                         0.9831     35069
   macro avg     0.9819    0.9793    0.9806     35069
weighted avg     0.9831    0.9831    0.9831     35069


Confusion matrix:
[[10847   353]
 [  238 23631]]


['mlp_attack_model.joblib']

In [ ]:
# Multiclass classification (atatck types)
X_multi = df[features]
y_multi = df['attack_cat_encoded']

X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(
    X_multi, y_multi, stratify=y_multi, test_size=0.2, random_state=42
)



# leaky, add hidden layer, add learning rate (reduce it),
# increase weightabe for some of the base classified instances,
# mor epre processing (add more feature using clustering),

from sklearn.utils.class_weight import compute_class_weight
classes = np.unique(y_train_m)
weights = compute_class_weight('balanced', classes = classes, y = y_train_m)
class_weights_dict = {cls: w for cls, w in zip(classes, weights)}

# sample_weights = compute_sample_weight(
#     class_weight=class_weights_dict,
#     y=y_train_m
# )

# sample_weights = y_train_m.map(class_weights_dict).values

# mlp_multi = Pipeline(steps=[
#     ('preproc', preprocessor),
#     ('clf', MLPClassifier(hidden_layer_sizes=(256, 128, 64), ## added 1 more hidden layer
#                           activation='relu', # changed relu -> leaky -> if needed -> TensorFlow/Keras
#                           solver='adam',
#                           alpha = 0.0001, # added weight decay for regularization
#                           learning_rate_init = 0.0005, # added learning rate
#                           max_iter=400,
#                           random_state=42,
#                           early_stopping=True))
# ])

# v0: accuracy = 0.8441
# This happens because MLP will always overfit majoriy classes unless we fix hte imbalance
# change 1: Add class weights, added learing rate, added alpha for weight decay, added 1 more hidden layer,



# v1: changedactivation from relu -> leaky & added learning rate

# mlp_multi.fit(X_train_m, y_train_m)
## this line had problems =? MLP DOESN'T Support class weights (the fit unction) -> use Keras



In [ ]:
# Keras model can't accept raw strings so, we wil have to do some pre processing
X_train_num = preprocessor.fit_transform(X_train_m)
X_test_num = preprocessor.transform(X_test_m)

# we need to use transform ony on test data nad fit transform on training data




In [ ]:
# MLP wasn't working wth weights so we go t Keras
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers

num_classes = len(np.unique(y_train_m))

# The actual number of features after preprocessing (195)
input_dim = X_train_num.shape[1]

# model = models.Sequential([
#     # Use the Input layer, which is the Keras best practice
#     layers.Input(shape=(input_dim,)),

#     layers.Dense(128, activation='leaky_relu'),
#     layers.BatchNormalization(),
#     layers.Dropout(0.3),

#     layers.Dense(64, activation='leaky_relu'),
#     layers.BatchNormalization(),
#     layers.Dropout(0.3),

#     layers.Dense(num_classes, activation='softmax')
# ])

# model = models.Sequential([  # TRY 2
#     # Use Input layer for clarity
#     layers.Input(shape=(input_dim,)),

#     # Smaller first layer: 64 neurons (was 128)
#     # Added L2 regularization to penalize large weights
#     layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
#     layers.BatchNormalization(),
#     layers.Dropout(0.3),

#     # Smaller second layer: 32 neurons (was 64)
#     layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
#     layers.BatchNormalization(),
#     layers.Dropout(0.3),

#     layers.Dense(num_classes, activation='softmax')
# ])

model = models.Sequential([  # try 3
    layers.Input(shape=(input_dim,)),

                             # Layer 0.1: Wider, 128 neurons
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    # Layer 1: Wider, 128 neurons
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    # Layer 2: 64 neurons
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    # Layer 3: New, 32 neurons (Slightly deeper model)
    layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(num_classes, activation='softmax')
])
# model.compile(
#     # ... rest of your compile arguments
# )

# # ... proceed with model.fit() using X_train_num

# model = models.Sequential([
#     layers.Dense(128, activation='leaky_relu', input_shape=(X_train_m.shape[1],)),
#     layers.BatchNormalization(),
#     layers.Dropout(0.3),

#     layers.Dense(64, activation='leaky_relu'),
#     layers.BatchNormalization(),
#     layers.Dropout(0.3),

#     layers.Dense(num_classes, activation='softmax')
# ])

# model.compile(
#     optimizer=tf.keras.optimizers.AdamW(learning_rate=0.001),
#     loss='sparse_categorical_crossentropy',
#     metrics=['accuracy']
# )

# model.fit(
#     X_train_num,
#     y_train_m,
#     epochs=30,
#     batch_size=64,
#     class_weight=class_weights_dict,# This actually works here
#     validation_split=0.2,
#     callbacks=[
#         tf.keras.callbacks.EarlyStopping(
#             monitor='val_loss',
#             patience=5,
#             restore_best_weights=True
#          )
#     ]
# )

# New callback to reduce learning rate when val_loss stops improving
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,           # Reduce learning rate by 50%
    patience=3,           # Wait 3 epochs before reducing
    min_lr=1e-6,
    verbose=1
)

model.compile(
    # Keep AdamW, but ensure learning rate is explicit
    optimizer=tf.keras.optimizers.AdamW(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(
    X_train_num,
    y_train_m,
    epochs=50, # Increase epochs since we are using more patience/regularization
    batch_size=64,
    class_weight=class_weights_dict,
    validation_split=0.2,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True), # Increased patience
        lr_scheduler # Added LR scheduler
    ]
)

Epoch 1/50
1754/1754 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - accuracy: 0.4924 - loss: 1.8745 - val_accuracy: 0.6130 - val_loss: 1.0215 - learning_rate: 0.0010
Epoch 2/50
1754/1754 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.6122 - loss: 1.3599 - val_accuracy: 0.6274 - val_loss: 0.9132 - learning_rate: 0.0010
Epoch 3/50
1754/1754 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.6367 - loss: 1.2839 - val_accuracy: 0.6448 - val_loss: 0.8777 - learning_rate: 0.0010
Epoch 4/50
1754/1754 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.6498 - loss: 1.2208 - val_accuracy: 0.6401 - val_loss: 0.8778 - learning_rate: 0.0010
Epoch 5/50
1754/1754 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.6481 - loss: 1.2874 - val_accuracy: 0.6635 - val_loss: 0.8824 - learning_rate: 0.0010
Epoch 6/50
1754/1754 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.6576 - loss: 1.2322 - val_accuracy: 0.6987 - val_loss: 0.8489 - learning_rate: 0.0010
Epoch 7/50
1754/1754 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.6

In [ ]:
# y_pred_multi = mlp_multi.predict(X_test_m)

# print(classification_report(y_test_m, y_pred_multi, digits=4))

# 1. Predict using the Keras model (not mlp_multi)
#    Input must be the preprocessed test data (X_test_num)
y_pred_probs = model.predict(X_test_num)

# 2. Convert probabilities to class labels (0, 1, 2...) using argmax
y_pred_classes = np.argmax(y_pred_probs, axis=1)

# 3. Generate report
print(classification_report(y_test_m, y_pred_classes, digits=4))

1096/1096 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
              precision    recall  f1-score   support

           0     0.2181    0.3200    0.2594       400
           1     0.0461    0.5186    0.0847       349
           2     0.3274    0.2780    0.3007      2453
           3     0.8783    0.4107    0.5597      6679
           4     0.7151    0.8331    0.7696      3637
           5     0.9999    0.9781    0.9889      8000
           6     0.9997    0.8966    0.9454     11200
           7     0.7728    0.7555    0.7640      2098
           8     0.2950    0.7225    0.4189       227
           9     0.0394    0.9615    0.0756        26

    accuracy                         0.7529     35069
   macro avg     0.5292    0.6675    0.5167     35069
weighted avg     0.8628    0.7529    0.7872     35069



In [ ]:
# kres use!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# hot encode for categorical data??!!!!!!!!!!!!
